과제W12_20150962_채현종

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
print(tf.__version__)

2.3.0


## 1.2019년 가계복지데이터를 불러들입니다.(5점)
c.koweps1.csv

In [2]:
df = pd.read_csv('C:/Users/yssg9187/Desktop/20-2/3. 응용자료분석/D_data/b_koweps01.csv')

## 2.성별더미, 흡연여부더미, 직업코드더미, 종교유무 더미를 만듭니다.(5점)

In [3]:
sex_dummies = pd.get_dummies(df['sex']).rename(columns=lambda x: 's'+str(x))
smoking_dummies = pd.get_dummies(df['smoking']).rename(columns=lambda x: 'k'+str(x))
job_dummies = pd.get_dummies(df['job_code']).rename(columns=lambda x: 'd'+str(x))
religion_dummies = pd.get_dummies(df['religion']).rename(columns=lambda x: 'r'+str(x))

## 3. 더미 변수를 포함한 데이터를 구성합니다.(5점)

In [4]:
df1 = pd.concat([df, religion_dummies, sex_dummies, smoking_dummies, job_dummies], axis=1)

In [5]:
df1

,Unnamed: 0,sex,religion,income,education_year,age,marriage_type,job_code,smoking,r1,...,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10
0,0,1,1,108.9,16,78,2,7,2,1,...,0,0,0,0,0,0,1,0,0,0
1,1,2,1,20.0,6,80,3,9,2,1,...,0,0,0,0,0,0,0,0,1,0
2,2,1,2,322.0,16,42,1,3,1,0,...,0,0,1,0,0,0,0,0,0,0
3,3,2,2,120.0,16,45,1,2,2,0,...,0,1,0,0,0,0,0,0,0,0
4,4,1,1,300.0,16,45,3,2,2,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,4502,1,1,230.0,12,54,1,8,2,1,...,0,0,0,0,0,0,0,1,0,0
4503,4503,2,1,138.0,16,58,1,8,2,1,...,0,0,0,0,0,0,0,1,0,0
4504,4504,2,2,286.0,12,27,3,4,2,0,...,0,0,0,1,0,0,0,0,0,0
4505,4505,1,1,179.0,12,64,1,8,2,1,...,0,0,0,0,0,0,0,1,0,0


## 4. 소득(income), 나이(age), 교육연수(education), 성별더미(sex), 흡연여부더미(smokint), 종교여부더미(region), 직업코드더미(job) X 변수로, 결혼여부를 Y 변수로 둡니다..(5점)

In [6]:
x_data =df1[['income', 'age', 'education_year', 's2', 'k2', 'r2', 'd2','d3','d4','d5','d6','d7','d8','d9','d10']].to_numpy()
y_data =df1[['marriage_type']].to_numpy()

## 5. tensorflow 자료 입출력 api를 이용한 dataset을 구축합니다.(5점)

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

## 6 x data와 y data를 동일한 숫자형으로 바꾸는 함수를 작성합니다..(float32)(10점)

In [8]:
def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

## 7. 넓이(width)는 10, 깊이(depth)는 5인 deep learning 모델을 구축합니다. (20점)

In [9]:
nb_classes = 10   # 넓이 = 10

In [10]:
k = x_data.shape[1]
k

15

In [11]:
# 신경이 5개 -> depth = 5

W1 = tf.Variable(tf.random.normal((k, nb_classes)), name='weight1')
b1 = tf.Variable(tf.random.normal((nb_classes,)), name='bias1')

W2 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight2')
b2 = tf.Variable(tf.random.normal((nb_classes,)), name='bias2')

W3 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight3')
b3 = tf.Variable(tf.random.normal((nb_classes,)), name='bias3')

W4 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight4')
b4 = tf.Variable(tf.random.normal((nb_classes,)), name='bias4')

W5 = tf.Variable(tf.random.normal((nb_classes, 1)), name='weight5')
b5 = tf.Variable(tf.random.normal((1,)), name='bias5')

In [12]:
def deep_nn(features):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)
    layer4 = tf.sigmoid(tf.matmul(layer3, W4) + b4)
    hypothesis = tf.sigmoid(tf.matmul(layer4, W5) + b5)
    return hypothesis

In [13]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001)

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(deep_nn(features),labels)
    return tape.gradient(loss_value, [W1, W2, W3, W4, W5, b1, b2, b3, b4, b5])

## 8. 10,000번 실행시키고 1000번마다 loss 값을 보이고, 10000번째 accuracy값을 보입니다. (15점)

In [14]:
EPOCHS = 10000

for step in range(EPOCHS + 1):
    for features, labels  in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(deep_nn(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W1, W2, W3, W4, W5, b1, b2, b3, b4, b5]))
        if step % 1000 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(deep_nn(features),labels)))
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(deep_nn(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Iter: 0, Loss: 3.5402
Iter: 1000, Loss: 2.5081
Iter: 2000, Loss: 1.6508
Iter: 3000, Loss: 0.9563
Iter: 4000, Loss: 0.3993
Iter: 5000, Loss: -0.0523
Iter: 6000, Loss: -0.4280
Iter: 7000, Loss: -0.7507
Iter: 8000, Loss: -1.0370
Iter: 9000, Loss: -1.2986
Iter: 10000, Loss: -1.5433
Testset Accuracy: 0.6450
